In [1]:
# Initial imports
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Loading data
file_path = Path("Resources/AHM_cleaned.csv")
AHM_cleaned_df = pd.read_csv(file_path)
AHM_cleaned_df.head()

,zpid,zipcode,latitude,longitude,latestPrice,livingAreaSqFt,latest_salemonth,latest_saleyear
0,111373431,78660,30.430632,-97.663078,305000.0,2601.0,9,2019
1,120900430,78660,30.432673,-97.661697,295000.0,1768.0,10,2020
2,2084491383,78660,30.409748,-97.639771,256125.0,1478.0,7,2019
3,120901374,78660,30.432112,-97.661659,240000.0,1678.0,8,2018
4,60134862,78660,30.437368,-97.656860,239900.0,2132.0,10,2018


In [3]:
# Drop zpid column since it has no impact on the target
AHM_cleaned_df = AHM_cleaned_df.drop('zpid', axis = 1)
AHM_cleaned_df.head()

,zipcode,latitude,longitude,latestPrice,livingAreaSqFt,latest_salemonth,latest_saleyear
0,78660,30.430632,-97.663078,305000.0,2601.0,9,2019
1,78660,30.432673,-97.661697,295000.0,1768.0,10,2020
2,78660,30.409748,-97.639771,256125.0,1478.0,7,2019
3,78660,30.432112,-97.661659,240000.0,1678.0,8,2018
4,78660,30.437368,-97.656860,239900.0,2132.0,10,2018


In [5]:
# Check data types for columns
AHM_cleaned_df.dtypes

zipcode               int64
latitude            float64
longitude           float64
latestPrice         float64
livingAreaSqFt      float64
latest_salemonth      int64
latest_saleyear       int64
dtype: object

In [6]:
# Define features set (delete the target column in this case "bad") 
# X is the input
X = AHM_cleaned_df.copy()
X = X.drop("latestPrice", axis=1)
X.head()

,zipcode,latitude,longitude,livingAreaSqFt,latest_salemonth,latest_saleyear
0,78660,30.430632,-97.663078,2601.0,9,2019
1,78660,30.432673,-97.661697,1768.0,10,2020
2,78660,30.409748,-97.639771,1478.0,7,2019
3,78660,30.432112,-97.661659,1678.0,8,2018
4,78660,30.437368,-97.656860,2132.0,10,2018


In [7]:
# Define target vector
y = AHM_cleaned_df["latestPrice"].values.reshape(-1, 1)
y[:5]

array([[305000.],
       [295000.],
       [256125.],
       [240000.],
       [239900.]])

In [8]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(11378, 6)
(3793, 6)
(11378, 1)
(3793, 1)


In [10]:
# Splitting into Train and Test sets into an 80/20 split.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)

In [11]:
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(12136, 6)
(3035, 6)
(12136, 1)
(3035, 1)


In [12]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [13]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [14]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the Decision Tree Model

In [15]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [16]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [17]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)
predictions

array([399900., 425000., 348000., ..., 497500., 350000., 289900.])

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Show the confusion matrix
cm_df.head()

In [18]:
# Calculating the accuracy score
#(True Positives (TP) + True Negatives (TN)) / Total = (51 + 20)/125 = 0.568
acc_score = accuracy_score(y_test, predictions)
acc_score

0.012918534141840231

In [ ]:
# Displaying results
# Precision for good loan apps = TP/(TP+FP)
# Precision for bad loan apps = TN/(TN+FN)
# Recall/Sensitivity for good loan apps = TP/(TP+FN)
# Recall/Sensitivity for bad loan apps = TN/(TN+FP)
# F1 Score is a weighted average of TP rate (recall) & precision between 0-1
# Support is the number of actual occurrences of the class in the dataset
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Create DOT data
dot_data = tree.export_graphviz(
    model, out_file=None, feature_names=X.columns, class_names=["0", "1"], filled=True
)

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())